---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [89]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    towns = pd.read_table("university_towns.txt",header=None,names=["RegionName"])
    #print(towns)
    current_state = ""
    def get_state(cell):
        if cell.endswith("[edit]"):
            global current_state
            current_state = cell[:-6]
            #print(current_state)
            return cell[:-6]
        else:
            return current_state

    towns["State"] = towns["RegionName"].map(get_state)
    
    
    towns = towns[~towns["RegionName"].str.endswith("[edit]")]
    #print(towns)
    towns["RegionName"] = towns["RegionName"].map(lambda x:x.split("(")[0].strip())
    towns = towns.reindex(columns=["State","RegionName"]).reset_index(drop=True)
    #towns.State = towns.State.map(dict(zip(states.values(),states.keys())))

    return towns
get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [99]:
gdp = pd.read_excel("gdplev.xls",header=1)[['Quarterly','Unnamed: 6']] \
    .rename(columns={"Unnamed: 6":"GDP"})\
    .iloc[217:] \
    .reset_index(drop=True)
       
gdp['GDP'] = gdp['GDP'].astype(np.float64)

gdp['Delta'] = gdp['GDP'].rolling(2).apply(lambda x:x[1]-x[0])
def start_end(x):
    if (x[0] < 0) &(x[1] < 0) & (x[2] > 0) & (x[3] > 0):
        return 1
    elif (x[1] > 0) & (x[2] < 0) & (x[3] < 0):
        return 0
    else:
        return np.nan 
gdp['Start'] = gdp['Delta'].rolling(4).apply(start_end)

In [100]:
gdp[gdp['Start'] >= 0.0]

,Quarterly,GDP,Delta,Start
35,2008q4,14577.0,-314.6,0.0
39,2009q4,14541.9,139.4,1.0


In [113]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    return gdp.loc[gdp[gdp['Start'] == 0.0].head(1).index[0] - 1]['Quarterly']
get_recession_start()

'2008q3'

In [114]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
       
    return gdp[gdp['Start'] == 1.0].head(1)["Quarterly"].values[0]
get_recession_end()

'2009q4'

In [115]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    start = get_recession_start()
    end = get_recession_end()
    regdp = gdp[(gdp['Quarterly']>start) & (gdp['Quarterly']<end)]
    return regdp[regdp['GDP'] == min(regdp['GDP'])].head(1)["Quarterly"].values[0]
get_recession_bottom()

'2009q2'

In [139]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    house = pd.read_csv("City_Zhvi_AllHomes.csv")
    ihouse = house.loc[:,"2000-01":"2016-08"]
    ihouse.columns = pd.to_datetime(ihouse.columns).to_period(freq="M")
    ghouse = ihouse.groupby(ihouse.columns.asfreq("Q"),axis=1).sum()
    house = pd.merge(house.loc[:,"RegionID":"SizeRank"],ghouse,left_index=True,right_index=True,how="inner")
    house.State = house.State.map(states)
    df = house.set_index(["State","RegionName"]).iloc[:,4:71]
    df = df.groupby(level=(0,1)).first()
    return df
convert_housing_data_to_quarters()

2000Q1    2000Q2    2000Q3    2000Q4    2001Q1  \
State     RegionName                                                          
Alabama   Adamsville       207100.0  207500.0  209400.0  215900.0  220400.0   
          Alabaster        366400.0  369200.0  369500.0  371100.0  369700.0   
          Albertville      221900.0  217800.0  218500.0  222600.0  227700.0   
          Arab             251300.0  244700.0  244000.0  248900.0  252600.0   
          Ardmore               NaN       NaN       NaN       NaN       NaN   
          Axis                  NaN       NaN       NaN       NaN       NaN   
          Baileyton             NaN       NaN       NaN       NaN       NaN   
          Bay Minette      245100.0  235600.0  237400.0  243900.0  257100.0   
          Bayou La Batre   132200.0  133500.0  132800.0  131000.0  127500.0   
          Bessemer              NaN       NaN       NaN       NaN       NaN   
          Birmingham       162100.0  163200.0  164900.0  168200.0  170500.0   
          Boaz             212600.0  210800.0  210900.0  214400.0  218500.0   
          Brent            278800.0  283000.0  288500.0  295000.0  289600.0   
          Brighton              NaN       NaN       NaN       NaN       NaN   
          Brookwood        277700.0  285300.0  296600.0  299900.0  305000.0   
          Buhl             272400.0  283800.0  289500.0  289700.0  297800.0   
          Calera           326800.0  331100.0  324000.0  323800.0  328000.0   
          Center Point     242900.0  243700.0  244500.0  246700.0  250100.0   
          Centreville      285900.0  289700.0  294000.0  298100.0  294300.0   
          Chalkville       282300.0  283300.0  283300.0  283300.0  288200.0   
          Chancellor            NaN       NaN       NaN       NaN       NaN   
          Chelsea          486200.0  501100.0  500700.0  502200.0  511900.0   
          Chickasaw        153600.0  161000.0  164800.0  168200.0  165400.0   
          Chunchula        240800.0  245300.0  246600.0  250200.0  256200.0   
          Citronelle       194500.0  199900.0  204200.0  203300.0  203600.0   
          Clay             362700.0  366800.0  371900.0  379500.0  384100.0   
          Coden            187800.0  194400.0  200600.0  205700.0  206800.0   
          Coker            354300.0  362300.0  354500.0  346000.0  342200.0   
          Concord          235800.0  236100.0  240400.0  248400.0  255400.0   
          Cottondale       302500.0  307900.0  314300.0  315900.0  320200.0   
...                             ...       ...       ...       ...       ...   
Wisconsin Vandenbroek           NaN       NaN       NaN       NaN       NaN   
          Vernon           549600.0  534600.0  522900.0  532400.0  558700.0   
          Vienna           534100.0  544600.0  547300.0  558900.0  571800.0   
          Vinland          359400.0  380300.0  404800.0  413500.0  432900.0   
          Wales                 NaN       NaN       NaN       NaN       NaN   
          Waterford        363600.0  358300.0  360600.0  365900.0  381000.0   
          Waukesha         423800.0  415300.0  410200.0  419500.0  442800.0   
          Waunakee         562200.0  574300.0  578000.0  581200.0  592000.0   
          Waupun           252000.0  252000.0  253200.0  253600.0  255300.0   
          Wausau           208700.0  208400.0  213100.0  209500.0  209600.0   
          Wayne                 NaN       NaN       NaN       NaN       NaN   
          West Allis       242800.0  245800.0  252200.0  242000.0  237300.0   
          Weston           240500.0  248100.0  252500.0  257400.0  261800.0   
          Wheatland        389400.0  396400.0  404800.0  413500.0  427700.0   
          Whitelaw         288100.0  304300.0  324100.0  342000.0  349000.0   
          Williams Bay     368700.0  345900.0  332300.0  332100.0  337800.0   
          Wilson           387100.0  388100.0  397300.0  411600.0  430600.0   
          Wind Lake        373100.0  371000.0  372400.0  378200.0  396400.0   


In [128]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    start = pd.Period(get_recession_start())
    bottom = pd.Period(get_recession_bottom())
    house = convert_housing_data_to_quarters().loc[:,[start,bottom]]
    house.columns = ["Start","Bottom"]
    house["Ratio"] = house.Start / house.Bottom
    house = house.dropna(axis=0,how="any")
    collage = get_list_of_university_towns().set_index(["State","RegionName"])
    collage["isUnv"] = "Yes"
    res = pd.merge(house,collage,how="left",left_index=True,right_index=True)
    res.isUnv = res.isUnv.fillna("No")

    res_u = res[res.isUnv == "Yes"].Ratio
    res_n = res[res.isUnv == "No"].Ratio
    #print(res_n)
    _,p = ttest_ind(res_u,res_n)
    different = (True if p < 0.01 else False)
    better = ("university town" if np.nanmean(res_u) < np.nanmean(res_n) else "non-university town")
    return different, p, better
run_ttest()

(True, 0.0059324820207098568, 'university town')

In [140]:
#convert_housing_data_to_quarters().loc["Texas"].loc["Austin"].loc["2010Q3"]